In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
data = pd.read_csv('merged_data.csv',encoding='utf-8')
data.head()

,Title,WikiID,Genres,Summary,sentiment
0,Ghosts of Mars,975900,"{""/m/01jfsb"": ""Thriller"", ""/m/06n90"": ""Science...","Set in the second half of the 22nd century, th...","{'neg': 0.15, 'neu': 0.786, 'pos': 0.065, 'com..."
1,White Of The Eye,9363483,"{""/m/01jfsb"": ""Thriller"", ""/m/0glj9q"": ""Erotic...",A series of murders of rich young women throug...,"{'neg': 0.198, 'neu': 0.724, 'pos': 0.078, 'co..."
2,A Woman in Flames,261236,"{""/m/07s9rl0"": ""Drama""}","Eva, an upper class housewife, becomes frustra...","{'neg': 0.065, 'neu': 0.835, 'pos': 0.1, 'comp..."
3,The Sorcerer's Apprentice,18998739,"{""/m/0hqxf"": ""Family Film"", ""/m/01hmnh"": ""Fant...","Every hundred years, the evil Morgana returns...","{'neg': 0.131, 'neu': 0.776, 'pos': 0.092, 'co..."
4,Little city,6631279,"{""/m/06cvj"": ""Romantic comedy"", ""/m/0hj3n0w"": ...","Adam, a San Francisco-based artist who works a...","{'neg': 0.108, 'neu': 0.804, 'pos': 0.089, 'co..."


In [6]:
data['sentiment'][1]

"{'neg': 0.198, 'neu': 0.724, 'pos': 0.078, 'compound': -0.9985}"

In [4]:
data['pos'] = data.apply(lambda row: row.sentiment['pos'], axis=1)
data.head()

TypeError: string indices must be integers